In [2]:
import google.generativeai as genai
import anthropic
gemini_api='your_api_key_here'
anthropic_api='your_api_key_here'
genai.configure(api_key=gemini_api)

In [3]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [4]:
class agent:
    """
    Represents an agent for playing tic-tac-toe.

    Attributes:
    - symbol: The symbol (X or O) representing the agent.
    - current_state: The current state of the tic-tac-toe board.
    - model_path: The path to the language model used by the agent.

    Methods:
    - model(): Returns the language model based on the model path.
    - get_prompt(): Returns the prompt for the agent to make a move.
    - generate(): Generates a move based on the language model.

    """

    def __init__(self, symbol, current_state, model_path):
        self.symbol = symbol
        self.current_state = current_state
        self.model_path = model_path

    def model(self):
        """
        Returns the language model based on the model path.

        Returns:
        - The language model object.

        """
        if self.model_path == "gemini-pro":
            return genai.GenerativeModel('gemini-pro')
        elif self.model_path == "claude-3-opus-20240229":
            return anthropic.Anthropic(api_key=anthropic_api)

    def get_prompt(self):
        """
        Returns the prompt for the agent to make a move.

        Returns:
        - The prompt string.

        """
        return f''' The current state of the tic-tac-toe board is as follows:
        {self.current_state[0]} | {self.current_state[1]} | {self.current_state[2]}
        ---------
        {self.current_state[3]} | {self.current_state[4]} | {self.current_state[5]}
        ---------
        {self.current_state[6]} | {self.current_state[7]} | {self.current_state[8]}
The place where a number is present there you can place your symbol {self.symbol}.You can place your symbol by giving the position number from 0 to 8. For example if you want to place your symbol at the top left corner then you can give 0 as input. If you want to place your symbol at the bottom right corner then you can give 8 as input.
You should only return the position number where you want to place your symbol.Follow the instruction otherwise a cat will die. Win the game to save the cat.Return only the position number where you want to place your symbol.'''

    def generate(self):
        """
        Generates a move based on the language model.

        Returns:
        - The generated move.

        """
        if self.model_path == "gemini-pro":
            return self.model().generate_content(self.get_prompt(), safety_settings=safety_settings).text
        elif self.model_path == "claude-3-opus-20240229":
            return self.model().messages.create(model="claude-3-opus-20240229", messages=[{"role": "user", "content": self.get_prompt()}], max_tokens=1000, temperature=0.0).content[0].text


In [5]:
class tic_tac_toe:
    """
    Represents a Tic Tac Toe game.

    Attributes:
    - current_state: A list representing the current state of the game board.
    - player1: An instance of the agent class representing player 1.
    - player2: An instance of the agent class representing player 2.
    """

    def __init__(self):
        """
        Initializes a new instance of the tic_tac_toe class.
        """
        self.current_state = ['0','1','2','3','4','5','6','7','8']
        self.player1 = agent('X',self.current_state,'gemini-pro')
        self.player2 = agent('O',self.current_state,'gemini-pro')# Change to claude-3-opus-20240229 to see a battle

    def print_board(self):
        """
        Prints the current state of the game board.
        """
        print(f'''{self.current_state[0]} | {self.current_state[1]} | {self.current_state[2]}
-----------
{self.current_state[3]} | {self.current_state[4]} | {self.current_state[5]}
-----------
{self.current_state[6]} | {self.current_state[7]} | {self.current_state[8]}''')

    def check_winner(self):
        """
        Checks if there is a winner in the current state of the game.

        Returns:
        - True if there is a winner, False otherwise.
        """
        if self.current_state[0] == self.current_state[1] == self.current_state[2] or self.current_state[3] == self.current_state[4] == self.current_state[5] or self.current_state[6] == self.current_state[7] == self.current_state[8] or self.current_state[0] == self.current_state[3] == self.current_state[6] or self.current_state[1] == self.current_state[4] == self.current_state[7] or self.current_state[2] == self.current_state[5] == self.current_state[8] or self.current_state[0] == self.current_state[4] == self.current_state[8] or self.current_state[2] == self.current_state[4] == self.current_state[6]:
            return True
        else:
            return False

    def check_draw(self):
        """
        Checks if the game is a draw.

        Returns:
        - True if the game is a draw, False otherwise.
        """
        if 'X' not in self.current_state and 'O' not in self.current_state:
            return True
        else:
            return False

    def get_appropriate_position(self,player):
        """
        Gets the appropriate position for the player to make a move.

        Parameters:
        - player: An instance of the agent class representing the player.

        Returns:
        - The position where the player should make a move.
        """
        while True:
            position = int(player.generate())
            if self.current_state[position] != 'X' and self.current_state[position] != 'O':
                return position
            else:
                continue

    def play(self):
        """
        Starts the Tic Tac Toe game and allows players to make moves until there is a winner or a draw.
        """
        self.print_board()
        while True:
            position = self.get_appropriate_position(self.player1)
            if self.current_state[position] != 'X' and self.current_state[position] != 'O':
                self.current_state[position] = 'X'
                self.print_board()
                self.player1.current_state = self.current_state
                if self.check_winner():
                    print('Player1 wins')
                    break
                if self.check_draw():
                    print('Match draw')
                    break
            else:
                continue
            position = self.get_appropriate_position(self.player2)
            if self.current_state[position] != 'X' and self.current_state[position] != 'O':
                self.current_state[position] = 'O'
                self.print_board()
                self.player2.current_state = self.current_state
                if self.check_winner():
                    print('Player2 wins')
                    break
                if self.check_draw():
                    print('Match draw')
            else:
                continue
        self.print_board()


In [6]:
tic_tac_toe().play()

0 | 1 | 2
-----------
3 | 4 | 5
-----------
6 | 7 | 8
0 | 1 | 2
-----------
3 | X | 5
-----------
6 | 7 | 8
0 | 1 | 2
-----------
3 | X | O
-----------
6 | 7 | 8
0 | 1 | 2
-----------
3 | X | O
-----------
X | 7 | 8
0 | O | 2
-----------
3 | X | O
-----------
X | 7 | 8
0 | O | 2
-----------
3 | X | O
-----------
X | X | 8
0 | O | O
-----------
3 | X | O
-----------
X | X | 8
0 | O | O
-----------
3 | X | O
-----------
X | X | X
Player1 wins
0 | O | O
-----------
3 | X | O
-----------
X | X | X
